# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!


  1. Use RAGAS to Generate Synthetic Data
  2. Load them into a LangSmith Dataset
  3. Evaluate our RAG chain against the synthetic test data
  4. Make changes to our pipeline
  5. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [1]:
#!pip install -qU ragas==0.2.10

In [2]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [3]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify
[nltk_data]     failed: unable to get local issuer certificate
[nltk_data]     (_ssl.c:1000)>


False

In [4]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [5]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"PSI - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [6]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [7]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "bills/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [8]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [9]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [10]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [11]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying SummaryExtractor:   0%|          | 0/20 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/20 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/60 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 20, relationships: 222)

We can save and load our knowledge graphs as follows.

In [12]:
kg.save("bills/ai_law.json")
bills_data_kg = KnowledgeGraph.load("bills/ai_law.json")
bills_data_kg

KnowledgeGraph(nodes: 20, relationships: 222)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [13]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=bills_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [14]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

</div>


<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer :

* Single Hop Specific Query Synthesizer
* Multi Hop Abstract Query Synthesizer
* Multi Hop Specific Query Synthesizer
</span>


Finally, we can use our `TestSetGenerator` to generate our testset!

In [15]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What is the EU?,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,The European Union's landmark AI Act of 2024 h...,single_hop_specifc_query_synthesizer
1,What is ASI?,[AI presents enormous opportunities for the Ph...,AI presents enormous opportunities for the Phi...,single_hop_specifc_query_synthesizer
2,What does Artificial Intelligence System mean ...,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,The context does not explicitly define the ter...,single_hop_specifc_query_synthesizer
3,What is AGI?,"[1 \na) Promote innovation, technological adva...",The context states that the Act shall regulate...,single_hop_specifc_query_synthesizer
4,In the context of artificial intelligence syst...,[1\n2\n3\n4\n5\n6\n7\n8 \n9\n10\n11\n12\n13\n1...,Foundation models refer to a type of large-sca...,single_hop_specifc_query_synthesizer
5,HOW CAN THE PHILIPPINES ENSURE ETHICAL AND RES...,[<1-hop>\n\nTWENTIETH CONGRESS OF THE \nREPUBL...,THE CONTEXT SHOWS THAT THE PHILIPPINES IS WORK...,multi_hop_abstract_query_synthesizer
6,How do audit mechanisms and sanctions for non-...,[<1-hop>\n\n1 \nSec. 15. AI Ethics Review Boar...,The context specifies that the AI Ethics Revie...,multi_hop_abstract_query_synthesizer
7,How do Large Language Models (LLMs) contribute...,"[<1-hop>\n\n1 \na) Promote innovation, technol...",Large Language Models (LLMs) are a type of gen...,multi_hop_abstract_query_synthesizer
8,How do Sec. 15 and Sec. 16 of the AI Act relat...,[<1-hop>\n\n1 \nSec. 8. NAICSecretariat. - The...,Sec. 15 of the AI Act mandates the NAIC to est...,multi_hop_specific_query_synthesizer
9,WhaT is the main purpous of the AI Regulatoin ...,[<1-hop>\n\nTWENTIETH CONGRESS OF THE \nREPUBL...,"The AI Regulatoin Act (AIRA), as introdused in...",multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [16]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying SummaryExtractor:   0%|          | 0/20 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/20 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/60 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [17]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is the significance of the European Union...,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,The European Union AI Act of 2024 is highlight...,single_hop_specifc_query_synthesizer
1,What is the importance of 3 in AI regulation a...,[AI presents enormous opportunities for the Ph...,AI presents opportunities and risks for the Ph...,single_hop_specifc_query_synthesizer
2,Can you explain the role of the Philippines in...,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,The Philippines recognizes that science and te...,single_hop_specifc_query_synthesizer
3,What is AGI supposed to do in AI development?,"[1 \na) Promote innovation, technological adva...",The context states that AI systems should enha...,single_hop_specifc_query_synthesizer
4,How do the provisions for algorithmic audits f...,[<1-hop>\n\n1 \nSec. 15. AI Ethics Review Boar...,The provisions in Sec. 15 establish that the A...,multi_hop_abstract_query_synthesizer
5,How do neural nerts inspired by the human brai...,[<1-hop>\n\n1 \nground its responses in verifi...,"Neural nerts, which are computational models i...",multi_hop_abstract_query_synthesizer
6,What info need include in AI registration like...,[<1-hop>\n\n1 \nSec. 12. Contents of AI Regist...,The AI registration must include information s...,multi_hop_abstract_query_synthesizer
7,How do the roles of gov departments like DOST ...,[<1-hop>\n\n1 \nSec. 6. Jurisdiction of the NA...,"The NAIC, which includes the Secretary of DOST...",multi_hop_abstract_query_synthesizer
8,"How does the DOST's role in AI regulation, as ...","[<1-hop>\n\n1 \nendorsements, voice recordings...",The DOST is responsible for implementing penal...,multi_hop_specific_query_synthesizer
9,how AI superintelligence might be dangerous fo...,[<1-hop>\n\n1\n2\n3\n4\n5\n6\n7\n8 \n9\n10\n11...,The context explains that AI presents big oppo...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [18]:
from langsmith import Client

client = Client()

dataset_name = "Philippines AI Bills"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Philippines AI Bills"
)

LangSmithConflictError: Conflict for /datasets. HTTPError('409 Client Error: Conflict for url: https://api.smith.langchain.com/datasets', '{"detail":"Dataset with this name already exists."}')

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [ ]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [ ]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [ ]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Bills RAG"
)

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [ ]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [ ]:
rag_chain.invoke({"question" : "How much is the penalty for spreading disinformation?"})

'The penalty for using AI to create or disseminate disinformation is a fine of One Million Pesos (Php 1,000,000) to Five Million Pesos (Php 5,000,000), or imprisonment of three (3) years to ten (10) years, or both, at the discretion of the court.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [ ]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [ ]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### 🏗️ Question #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer:

* Answer here
</div>

## LangSmith Evaluation

In [32]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'kind-reward-29' at:
https://smith.langchain.com/o/fa493a47-99f1-4723-a121-32a4e602af4a/datasets/331d0c73-4001-42a2-99ea-81861c19bd72/compare?selectedSessions=ab22537c-68e1-468f-b8a0-307ee5f42a3b




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How does the Philippines' AI policy framework ...,"Based on the provided context, the Philippines...",None,The Philippines' AI policy framework aims to s...,0,0,0,5.813875,f688e170-48ed-4f70-99b6-6c5bac34522b,ad0321e9-eeca-420e-8216-80bce992683f
1,How does the Philippines' Artificial Intellige...,The Philippines' Artificial Intelligence Regul...,None,The Philippines' Artificial Intelligence Regul...,1,0,0,3.287384,a583e7fd-4333-4ff2-8766-174a5f76fe4f,8cb61178-1d5a-4e50-bc48-accc1ac3ecae
2,what sec 10 and sec 12 say about AI registrati...,"Based on the provided context, there is no spe...",None,Sec 12 says that AI registration needs info li...,0,0,0,4.818289,c74bfaaf-92bc-4e5e-988a-f511d85418bb,a553952c-2b56-47fd-897b-bcf535f6d549
3,How does the development and regulation of Art...,"Based on the provided context, the national AI...",None,The Philippine AI policies recognize the impor...,1,1,0,5.918671,01b2d795-9eff-40cd-afa0-5e4814184e8b,f1ee8f5a-587e-4935-bd47-c409ea3acf06
4,How can AI policies balance inovation with saf...,"Based on the provided context, AI policies in ...",None,The context highlights that AI offers signific...,1,1,0,4.955808,66127324-4a15-4bc0-886d-582883e8976f,d43cefb8-b127-4b3b-a572-de0984979845
5,How does the AI regulation promote long-term r...,The AI regulation promotes long-term resilienc...,None,The AI Regulation Act (AIRA) promotes long-ter...,1,1,0,4.077708,7b40c63b-5479-4008-b44c-7c5f7b2d81cc,35d93a80-b982-4bfc-a466-42684e495741
6,How do the AI registration requirements outlin...,The AI registration requirements outlined in S...,None,The AI registration requirements in Sec. 12 sp...,1,1,0,3.743365,61254b00-35fd-4181-84b6-40ff209b860f,2c646fab-7345-4aca-9bea-05d6720d3124
7,How does the AI regulation act address the iss...,The AI regulation act addresses the issues of ...,None,The AI regulation act emphasizes the importanc...,1,1,0,4.032416,b43985da-e240-4ea7-8ca2-9cede2c506d0,af86fe89-3b8d-4f2a-80cd-499d428cc3b6
8,What role does Artificial General Intelligence...,"Based on the provided context, Artificial Gene...",None,The context states that the Act shall regulate...,1,1,0,3.444118,714e19b8-440f-4a1b-a4be-d0b182588da7,b86c60a3-8c15-49e2-ad9b-a7ec9a279fc0
9,How does the Philippine Constitution relate to...,I don't know.,None,"The context mentions Article XIV, Section 10 o...",0,0,0,1.704314,ff53cd67-90b1-4f69-a1cc-0aedc87ab096,627a859a-c784-4c90-8034-54f04d000816


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [33]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [34]:
rag_documents = docs

In [35]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer:

</div>

In [36]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer :

</div>

In [37]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Bills RAG 2"
)

In [38]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [39]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [40]:
empathy_rag_chain.invoke({"question" : "Why is the Philippines AI Bill important?"})

"Thank you for asking such an important question. Based on the context provided, the Philippines AI Bill is important because it aims to carefully balance encouraging technological innovation with ensuring that AI systems remain safe, ethical, transparent, and subject to meaningful human oversight. This bill recognizes that AI is still in its early stages and provides a general framework to promote responsible and lawful AI development.\n\nImportantly, it envisions a future where AI supports Filipino creativity and ingenuity, helps address national development challenges, and protects the rights and welfare of every citizen. The State takes on the responsibility to ensure that AI is never used to commit crimes, violate rights, or cause harm—whether intentionally or accidentally—while fostering technological progress.\n\nIn essence, this bill seeks to guide the Philippines towards an innovative yet secure and ethical digital future, keeping the well-being of the Filipino people at its h

Finally, we can evaluate the new chain on the same test set!

In [41]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'clear-house-13' at:
https://smith.langchain.com/o/fa493a47-99f1-4723-a121-32a4e602af4a/datasets/331d0c73-4001-42a2-99ea-81861c19bd72/compare?selectedSessions=7c735df5-d74e-49ba-9b43-fb7c6b1d7196




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How does the Philippines' AI policy framework ...,Thank you for your thoughtful question about h...,None,The Philippines' AI policy framework aims to s...,1,0,1,4.518684,f688e170-48ed-4f70-99b6-6c5bac34522b,695aec2a-6410-44f4-92b6-d6314f93e884
1,How does the Philippines' Artificial Intellige...,Thank you for your thoughtful question. From t...,None,The Philippines' Artificial Intelligence Regul...,1,1,1,4.616806,a583e7fd-4333-4ff2-8766-174a5f76fe4f,e963d343-a18c-4a71-9b35-200d26edde39
2,what sec 10 and sec 12 say about AI registrati...,Thank you for your thoughtful question. I unde...,None,Sec 12 says that AI registration needs info li...,0,0,1,5.229622,c74bfaaf-92bc-4e5e-988a-f511d85418bb,bcf47014-e961-4e13-bdc5-11d629866bb8
3,How does the development and regulation of Art...,Thank you for your thoughtful question. Based ...,None,The Philippine AI policies recognize the impor...,1,1,1,5.657937,01b2d795-9eff-40cd-afa0-5e4814184e8b,a4ba107a-58a5-44d8-9658-6e8be8837d16
4,How can AI policies balance inovation with saf...,Thank you for your thoughtful question. It's c...,None,The context highlights that AI offers signific...,1,1,1,4.881674,66127324-4a15-4bc0-886d-582883e8976f,7fe5dead-e74f-4554-b433-f033407f1cc1
5,How does the AI regulation promote long-term r...,Thank you for your thoughtful question. Based ...,None,The AI Regulation Act (AIRA) promotes long-ter...,1,1,1,3.192215,7b40c63b-5479-4008-b44c-7c5f7b2d81cc,62881107-2a68-4b91-8e20-21b4f27eb4b3
6,How do the AI registration requirements outlin...,Thank you for your thoughtful question. Based ...,None,The AI registration requirements in Sec. 12 sp...,1,1,1,3.739093,61254b00-35fd-4181-84b6-40ff209b860f,4bf11002-0f73-4d47-8255-37267d41e955
7,How does the AI regulation act address the iss...,Thank you for your thoughtful question. Based ...,None,The AI regulation act emphasizes the importanc...,1,1,1,3.935096,b43985da-e240-4ea7-8ca2-9cede2c506d0,175a25d9-5f33-4577-aa23-6e2e200b3d66
8,What role does Artificial General Intelligence...,Thank you for your thoughtful question. Based ...,None,The context states that the Act shall regulate...,0,0,1,4.360935,714e19b8-440f-4a1b-a4be-d0b182588da7,64f15a46-e457-4ebe-9610-0b3b8a50eddc
9,How does the Philippine Constitution relate to...,Thank you for your thoughtful question. Based ...,None,"The context mentions Article XIV, Section 10 o...",1,1,1,3.676950,ff53cd67-90b1-4f69-a1cc-0aedc87ab096,608ef7da-6ad2-4151-ad98-aa18aa1a14dd


<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### 🏗️ Question #4:

Explain why you believe certain metrics changed in certain ways, and provide a screenshot of the difference between the two chains.

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer:

</div>